## Test OWL BART

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.calibration import calibration_curve
# from bart_playground import *
from bart_playground.bart import DefaultBART, ProbitBART, OWLBART #, temp #, OWLBART
# from bart_playground.priors import OWLPrior


In [ ]:
# Parameters
N_TREES = 50
NDPOST = 1000
NSKIP = 200
RANDOM_STATE = 42

# If debug then run with only one dataset and record running time
# Otherwise run with all datasets
debug = False

In [ ]:
def simulate_data(n=30, seed=1):
    np.random.seed(seed)
    X = np.random.uniform(-1, 1, size=(n, 50))
    A = np.random.choice([-1, 1], size=n)
    base = 1 + 2*X[:, 0] + X[:, 1] + 0.5*X[:, 2]
    
    # Scenario 1
    T1 = 0.442 * (1 - X[:, 0] - X[:, 1]) * A
    mu1 = base + T1
    R1 = np.random.normal(loc=mu1, scale=1)

    # Scenario 2
    T2 = (X[:, 1] - 0.25 * (X[:, 0])**2 - 1) * A
    mu2 = base + T2
    R2 = np.random.normal(loc=mu2, scale=1)

    # Scenario 3
    T3 = (0.5 - (X[:, 0])**2 - (X[:, 1])**2) * ((X[:, 0])**2 + (X[:, 1])**2 - 0.3) * A
    mu3 = base + T3
    R3 = np.random.normal(loc=mu3, scale=1)

    # Scenario 4
    T4 = (1 - (X[:, 0])**3 + np.exp((X[:, 2])**2 + X[:, 4]) + 0.6 * X[:, 5] - (X[:, 6] + X[:, 7])**2) * A
    mu4 = base + T4
    R4 = np.random.normal(loc=mu4, scale=1)
    
    # Convert to DataFrames
    colnames = [f"X{i+1}" for i in range(50)]
    base_df = pd.DataFrame(X, columns=colnames)
    base_df["A"] = A

    df1 = base_df.copy()
    df1["R"] = R1

    df2 = base_df.copy()
    df2["R"] = R2

    df3 = base_df.copy()
    df3["R"] = R3

    df4 = base_df.copy()
    df4["R"] = R4

    return {"Scenario 1": df1, "Scenario 2": df2, "Scenario 3": df3, "Scenario 4": df4}

# Print first 5 rows of each scenario
simulated = simulate_data(n=30)
for name, df in simulated.items():
    print(f"\n{name} (first 5 rows):")
    print(df.head())


In [ ]:
df = simulated["Scenario 1"]
X = df[[col for col in df.columns if col.startswith("X")]].values
A = df["A"].values
R = df["R"].values

X_train, X_test, A_train, A_test, R_train, R_test = train_test_split(
    X, A, R, test_size=0.3, random_state=42
)

owlbart = OWLBART(n_trees=50, ndpost=1000, nskip=200, random_state=42,
                  treatment=A_train, reward=R_train
)
owlbart.fit(X_train, A_train)
A_opt = owlbart.predict_optimal_treatment(X_test)
R_pred = owlbart.predict_outcome(X_test, A_opt)
